In [ ]:
import pandas as pd

In [3]:
df = pd.read_csv("sampled_x_train.csv")  # Randomized sample of the big dataset

In [40]:
def cleaning(
    df: pd.DataFrame,
    piezo: bool = False,
    meteo: bool = False,
    hydro: bool = False,
    prelev: bool = False,
    insee: bool = False
) -> pd.DataFrame:
    # Define columns to exclude for each category
    excluded_piezo = [
        "piezo_station_department_code",
        "piezo_station_update_date",
        "piezo_station_department_name",
        "piezo_station_commune_code_insee",
        "piezo_station_pe_label",
        "piezo_station_bdlisa_codes",
        "piezo_station_bss_code",
        "piezo_station_commune_name",
        "piezo_station_bss_id",
        "piezo_bss_code",
        "piezo_continuity_name",
        "piezo_producer_code",
        "piezo_producer_name",
        "piezo_measure_nature_name",
    ]
    excluded_meteo = ["meteo_id", "meteo_radiation_IR", "meteo_name"]
    excluded_hydro = [
        "hydro_station_code",
        "hydro_status_label",
        "hydro_method_code",
        "hydro_method_label",
        "hydro_qualification_label",
    ]
    excluded_prelev = [
        "prelev_structure_code_0",
        "prelev_volume_0",
        "prelev_usage_label_0",
        "prelev_volume_obtention_mode_label_0",
        "prelev_longitude_0",
        "prelev_latitude_0",
        "prelev_commune_code_insee_0",
        "prelev_structure_code_1",
        "prelev_volume_1",
        "prelev_usage_label_1",
        "prelev_volume_obtention_mode_label_1",
        "prelev_longitude_1",
        "prelev_latitude_1",
        "prelev_commune_code_insee_1",
        "prelev_structure_code_2",
        "prelev_volume_2",
        "prelev_usage_label_2",
        "prelev_volume_obtention_mode_label_2",
        "prelev_longitude_2",
        "prelev_latitude_2",
        "prelev_commune_code_insee_2",
        "prelev_other_volume_sum",
    ]
    excluded_insee = ['prelev_commune_code_insee_0', 'prelev_commune_code_insee_1',
       'prelev_commune_code_insee_2', 'insee_%_agri', 'insee_pop_commune',
       'insee_med_living_level', 'insee_%_ind', 'insee_%_const']
    # Combine columns to drop based on the parameters
    columns_to_drop = []
    if piezo:
        columns_to_drop += excluded_piezo
    if meteo:
        columns_to_drop += excluded_meteo
    if hydro:
        columns_to_drop += excluded_hydro
    if prelev:
        columns_to_drop += excluded_prelev
    if insee:
        columns_to_drop += excluded_insee

    # Drop columns safely (ignore errors for missing columns)

    df_filtered = df.drop(columns=columns_to_drop, errors="ignore")

    return df_filtered

# Cleaning all redundant value

In [41]:
# Removing redundant columns
df_filtered = cleaning(df, True, True, True, True, True)
df_filtered.head(2)

,row_index,piezo_station_investigation_depth,piezo_station_altitude,piezo_station_longitude,piezo_station_latitude,piezo_measurement_date,piezo_obtention_mode,piezo_status,piezo_qualification,piezo_continuity_code,...,distance_piezo_meteo,hydro_observation_date_elab,hydro_observation_result_elab,hydro_status_code,hydro_qualification_code,hydro_longitude,hydro_latitude,hydro_hydro_quantity_elab,distance_piezo_hydro,piezo_groundwater_level_category
0,951052,28.2,102.0,3.854086,49.669113,2020-12-28,Valeur mesurée,Donnée contrôlée niveau 2,Correcte,2.0,...,9.330036,2020-12-28,3336.0,16,20,3.710233,49.689910,QmJ,10.605403,Low
1,1070593,350.0,66.0,0.313916,45.309031,2021-02-11,Valeur mesurée,Donnée contrôlée niveau 2,Correcte,2.0,...,5.443081,2021-02-11,2483.0,12,20,0.303494,45.303934,QmJ,0.992686,Very High


# Unifies longitude and latitude columns

In [ ]:
def unify_long_lat(df: pd.DataFrame, distance_threshold: int = 25) -> pd.DataFrame:
    """
    Unifies longitude and latitude into single columns if distance_piezo_hydro is below the threshold.

    Parameters:
        df (pd.DataFrame): The DataFrame containing the data.
        distance_threshold (int): The maximum distance for unification.

    Returns:
        pd.DataFrame: The modified DataFrame with unified longitude and latitude columns.
    """
    # Columns to exclude after processing
    excluding = [
        "piezo_station_latitude",
        "piezo_station_longitude",
        "hydro_longitude",
        "hydro_latitude",
        "meteo_longitude",
        "meteo_latitude",
        "distance_piezo_hydro",
        "distance_hydro_meteo",
    ]

    # Ensure the required column exists
    if "distance_piezo_hydro" in df.columns:
        # Create unified longitude and latitude where condition is met
        df.loc[df["distance_piezo_hydro"] < distance_threshold, "longitude"] = df[
            "piezo_station_longitude"
        ]
        df.loc[df["distance_piezo_hydro"] < distance_threshold, "latitude"] = df[
            "piezo_station_latitude"
        ]

    # Drop the excluded columns
    df = df.drop(columns=excluding, errors="ignore")

    return df

In [ ]:
df_filtered = unify_long_lat(df_filtered)

In [42]:
df_filtered.head()

,row_index,piezo_station_investigation_depth,piezo_station_altitude,piezo_station_longitude,piezo_station_latitude,piezo_measurement_date,piezo_obtention_mode,piezo_status,piezo_qualification,piezo_continuity_code,...,distance_piezo_meteo,hydro_observation_date_elab,hydro_observation_result_elab,hydro_status_code,hydro_qualification_code,hydro_longitude,hydro_latitude,hydro_hydro_quantity_elab,distance_piezo_hydro,piezo_groundwater_level_category
0,951052,28.20,102.00,3.854086,49.669113,2020-12-28,Valeur mesurée,Donnée contrôlée niveau 2,Correcte,2.0,...,9.330036,2020-12-28,3336.0,16,20,3.710233,49.689910,QmJ,10.605403,Low
1,1070593,350.00,66.00,0.313916,45.309031,2021-02-11,Valeur mesurée,Donnée contrôlée niveau 2,Correcte,2.0,...,5.443081,2021-02-11,2483.0,12,20,0.303494,45.303934,QmJ,0.992686,Very High
2,2044217,14.46,96.62,0.416863,47.514282,2022-02-12,Valeur mesurée,Donnée contrôlée niveau 2,Correcte,2.0,...,11.805022,2022-02-12,182.0,16,20,0.476992,47.603299,QmJ,10.878132,Average
3,3023095,42.00,90.00,1.086858,44.490808,2023-02-17,Valeur mesurée,Donnée contrôlée niveau 1,Correcte,2.0,...,9.626371,2023-02-17,134.0,16,20,1.016880,44.518940,QmJ,6.370417,Very Low
4,1437526,7.00,39.00,0.445087,44.229290,2021-06-29,Valeur mesurée,Donnée contrôlée niveau 2,Correcte,2.0,...,11.681980,2021-06-29,296.0,12,12,0.484460,44.266114,QmJ,5.157637,Average


In [21]:
df_filtered.select_dtypes(include=["object"]).columns

Index(['piezo_measurement_date', 'piezo_obtention_mode', 'piezo_status',
       'piezo_qualification', 'piezo_measure_nature_code', 'meteo_date',
       'hydro_observation_date_elab', 'hydro_hydro_quantity_elab',
       'insee_%_agri', 'insee_med_living_level', 'insee_%_ind',
       'insee_%_const', 'piezo_groundwater_level_category'],
      dtype='object')

In [31]:
date_columns = df_filtered.filter(like="date", axis=1).columns
df_date = df_filtered[date_columns]
df_date.head()

,piezo_measurement_date,meteo_date,hydro_observation_date_elab
0,2020-12-28,2020-12-28,2020-12-28
1,2021-02-11,2021-02-11,2021-02-11
2,2022-02-12,2022-02-12,2022-02-12
3,2023-02-17,2023-02-17,2023-02-17
4,2021-06-29,2021-06-29,2021-06-29


In [38]:
def unify_date(df: pd.DataFrame) -> pd.DataFrame:
    """
    Unifies date combine date in a single column.

    Parameters:
        df (pd.DataFrame): The DataFrame containing the data.

    Returns:
        pd.DataFrame: The modified DataFrame with unifies dates columns.
    """
    # Columns to exclude after processing
    excluding = ["piezo_measurement_date", "meteo_date", "hydro_observation_date_elab"]

    # Check and assign the first available date column
    if "piezo_measurement_date" in df.columns:
        df["date"] = df["piezo_measurement_date"]

    df = df.drop(columns=excluding, errors="ignore")

    return df

In [39]:
df_unify = unify_date(df_filtered)
df_unify.head()

,row_index,piezo_station_investigation_depth,piezo_station_altitude,piezo_obtention_mode,piezo_status,piezo_qualification,piezo_continuity_code,piezo_measure_nature_code,meteo_altitude,meteo_rain_height,...,hydro_hydro_quantity_elab,insee_%_agri,insee_pop_commune,insee_med_living_level,insee_%_ind,insee_%_const,piezo_groundwater_level_category,longitude,latitude,date
0,951052,28.20,102.00,Valeur mesurée,Donnée contrôlée niveau 2,Correcte,2.0,N,75,3.4,...,QmJ,0,139.0,20470,0,0,Low,3.854086,49.669113,2020-12-28
1,1070593,350.00,66.00,Valeur mesurée,Donnée contrôlée niveau 2,Correcte,2.0,0,166,2.0,...,QmJ,24.3,528.0,23880,2.7,51.4,Very High,0.313916,45.309031,2021-02-11
2,2044217,14.46,96.62,Valeur mesurée,Donnée contrôlée niveau 2,Correcte,2.0,N,107,0.0,...,QmJ,5,914.0,22900,7,32,Average,0.416863,47.514282,2022-02-12
3,3023095,42.00,90.00,Valeur mesurée,Donnée contrôlée niveau 1,Correcte,2.0,I,104,0.2,...,QmJ,23.9,340.0,24280,1.5,5.2,Very Low,1.086858,44.490808,2023-02-17
4,1437526,7.00,39.00,Valeur mesurée,Donnée contrôlée niveau 2,Correcte,2.0,0,55,3.0,...,QmJ,40.8,755.0,22610,12.2,6.1,Average,0.445087,44.229290,2021-06-29
